__Идея:__ апроксимируем рейтинг $R_{ij}$ как произведение двух эмбедингов - пользовательского и товарного 

Соотвественно, матрица рейтигнов апроксимируется как произведение двух матриц U и V

Выход алгоритма: два вектора эмюедингов U и V

Их скалярное произведение даёт прогноз рейтинга

Плюсы:
- очень быстро считается

Модификации:
- BlockALS

Минимизируем ошибку приближения
<img src="img/als.png" width=350>

Функция не выпуклая => использовать всякие градиентные методы не получится

Однако, если зафиксировать одну из двух переменных, задача упрощается и становится решаемой аналитически

Алгоритм
0. инициализируем эмбединги чем-то
1. фиксируем пользовательский эмбединг и оптимизируемся по товарному
2. фиксируем товарный эмбединг и оптимизируемся по пользовательскому
3. повторяем до сходимости

Ниже формальное описание

<img src="img/als2.png" width=600>

### Вывод шага алгоритма для U

Фиксируем матрицу V, оптимизируем коэффициенты матрицы U.

Строки матрицы рейтингов независимы => оптимизируем построчно. Это удобно, можно запускать параллельно.

У нас получается m задач линейной регрессии с $n$ кейсами в каждом - рейтинг будем прогнозировать как комбинацию k факторов, кодирующих пользователя

Например возьмем пользователя $U_i$ и посчитаем ошибку приближения:

$$J(U_i) = \sum_{j \in items} \bigg(R_{ij} - U_{ij} V_j^T\bigg)^2$$

Здесь $R_i$ - строка с рейтингами пользователя, $U_i$ - эмбединг пользователя, $V^T$ - эмбединг товара

Добавим $L_2$ регуляризацию

$$J(U_i) = \sum_j (R_{ij} - U_{ij} V^T)^2 + \lambda \sum_j || U_{ij} ||^2$$

Сумму квадратов можно представить векторно, если в вектор то, по чему суммируем: набор $R_{ij}$ обозначим за $R_i$, а столбцы $V_j$ за матрицу $V$)

$$J(U_i) = (R_i - U_i V^T) (R_i - U_i V^T)^T$$

Чтобы привести к более классической форме записи формулы регрессии, перенесем транспонирование внутрь:

$$J(U_i) = (R_i^T - V U_i^T)^T (R_i^T - V U_i^T)$$

Вспоминаем постановку МНК-регрессии с регуляризацией

$$J(U_i) = (y - X \beta)^T (y - X \beta) + \lambda X^TX$$

Видим, что это и есть наш функционал:

$$R^T=y$$
$$V=X$$
$$U^T=\beta$$

Тогда вспоминаем формулу для Ridge регрессии (выводить не будем)

$$\beta = ((X^TX)^{-1} + \lambda I)X^Ty$$

И подставляем наши переменные

$$U^T = ((V^TV)^{-1} + \lambda I)V^TR^T$$

Для получения U транспонируем

$$U = (RV)((V^TV)^{-1} + \lambda I)$$

### Вывод шага алгоритма для V

Все аналогично U